In [1]:
import os
import clip
import torch
from torchvision.datasets import CIFAR100

In [2]:
from PIL import Image
import requests
from transformers import AutoProcessor, BlipModel

# model = BlipModel.from_pretrained("Salesforce/blip-image-captioning-base")
# processor = AutoProcessor.from_pretrained("Salesforce/blip-image-captioning-base")

# import cv2
import os
from tqdm import tqdm
import math
import numpy as np
import os
import clip
import torch
import re

In [3]:
device  = 'cuda'

In [4]:
# Download the dataset
cifar100 = CIFAR100(root=os.path.expanduser("~/.cache"), download=True, train=False)

Files already downloaded and verified


In [5]:
params = {}
params['model_size'] = "ViT-B/32"
params['device'] = device if torch.cuda.is_available() else "cpu"

model, preprocess = clip.load(params['model_size'], device=params['device'])
model.eval()
model.requires_grad_(False)
print("CLIP model loaded")



import torch
class Dataset(torch.utils.data.Dataset):
    
    def __init__(self, images):
        self.images=images
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, index):
        X = self.images[index]
        return preprocess(X)

params = {'batch_size': 32, 'shuffle': False, 'num_workers':4}

CLIP model loaded


In [6]:
testset=[]
correctlabels=[]
for image,class_id in cifar100:
    testset.append(image)
    correctlabels.append(class_id)

In [7]:
text_inputs = torch.cat([clip.tokenize(f"a photo of a {c}") for c in cifar100.classes]).to(device)
text_features = model.encode_text(text_inputs).float()

In [8]:
test_set = Dataset(testset)

In [9]:
results = []
testloader = torch.utils.data.DataLoader(test_set, **params)
for idx, item in enumerate(tqdm(testloader,desc="Inference")):
    with torch.no_grad():
        item = item.to(device)
        outputs = model.encode_image(item).float()

    results.append(outputs)
    
    
img_features = torch.vstack(results[0:-1])   

Inference: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 313/313 [00:39<00:00,  7.85it/s]


In [10]:
img_features.shape, text_features.T.shape

(torch.Size([9984, 512]), torch.Size([512, 100]))

In [11]:
img_features /= img_features.norm(dim=-1, keepdim=True)

In [12]:
values,indices = (img_features.squeeze() @ text_features.T).softmax(dim=1).topk(1)

In [13]:
accuracy = {}
for idx,item in enumerate(indices):
    accuracy[idx] = 0
    if (item.item() == correctlabels[idx]):
        accuracy[idx] = 1
            
print(f"Top-1 accuracy is {sum(accuracy.values())/len(accuracy)}")

Top-1 accuracy is 0.47766426282051283


In [14]:
values,indices = (img_features.squeeze() @ text_features.T).softmax(dim=1).topk(5)

In [15]:
accuracy = {}
for idx,item in enumerate(indices):
    accuracy[idx] = 0
    for curr_item in item:
        if (curr_item.item() == correctlabels[idx]):
            accuracy[idx] = 1
            
print(f"Top-5 accuracy is {sum(accuracy.values())/len(accuracy)}")

Top-5 accuracy is 0.7237580128205128
